In [ ]:
#------------------------------------------------Please Read README file---------------------------------------------------

!pip install gspread==3.6.0

from IPython.display import clear_output
from google.colab import drive
from datetime import datetime
from tabulate import tabulate
from github import Github
import pandas as pd
import requests
import tempfile
import gspread
import json
import re

def Main():
    clear()
    div()
    charge()

def clear():
    clear_output(wait=True)
    print("")

def div():
    global url_input
    global sheet
    url_input = input("Introduce una URL: ")
    sheet = input("¿Cómo se llama la hoja en la que quieres sobrescribir? ")

    parts = url_input.split('/d/')
    part_useful = parts[1]

    values = part_useful.split('/')
    sheet_id = values[0]
    part_useful2 = values[1]

    values = part_useful2.split('=')
    sheet_index = values[1]

    credentials()
    verification()

    read(sheet_index, sheet_id)

def read(sheet_index, sheet_id):
    print("Cargando...")
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx&gid={sheet_index}"
    df = pd.read_excel(url,  engine='openpyxl')
    df_plain = convert_to_plain_text(df)
    ETL(df_plain)

def convert_to_plain_text(df):
    for column in df.columns:
        df[column] = df[column].astype(str)
    df = df.replace("nan", "", regex=True)
    return df

def form():
    print("Para salir y continuar introduce || 0021 ||")
    print("")
    print(tabulate(dfg.head(30), headers='keys', tablefmt='psql'))
    print("")

def verification():
    global sheet
    global gc
    global spreadsheet

    while True:
        try:
            clear()
            worksheet = spreadsheet.worksheet(sheet)
            break
        except gspread.exceptions.WorksheetNotFound:
            clear()
            print("No existe una hoja de cálculo llamada", sheet)
            sheet = input("¿Cómo se llama la hoja en la que quieres sobrescribir? ")
def ETL(df):
    global dfg
    dfg = df.drop_duplicates()
    dfg = dfg.fillna('-')
    clear()

    form()
    char = input("Introduce tu Formula: ")

    while char != "0021":
        try:
            if ',' in char:
                tag = 1
            else:
                tag = 0

            if char.startswith("del("):
                match = re.match(r"del\((.+),\s*(.+)\)", char)
                if tag == 1:
                    if match:
                        char_to_remove = match.group(1)
                        column_name = match.group(2).strip()
                        char_escaped = re.escape(char_to_remove)
                        try:
                            dfg[column_name] = dfg[column_name].str.replace(char_escaped, "", regex=True)
                        except:
                          break
                else:
                    char_to_remove = char.replace('del(', '')
                    char_to_remove = char_to_remove.replace(')', '')
                    char_escaped = re.escape(char_to_remove).replace(r'\+', '+')
                    dfg = dfg.applymap(lambda x: re.sub(char_escaped, '', str(x)))
            elif char.startswith("space("):
                match = re.match(r"space\((.+),\s*(.+)\)", char)
                if tag == 1:
                    if match:
                        char_to_replace = match.group(1)
                        column_name = match.group(2).strip()
                        char_escaped = re.escape(char_to_replace).replace(r'\+', '+')
                        try:
                            dfg[column_name] = dfg[column_name].str.replace(char_escaped, " ", regex=True)
                        except:
                            break
                else:
                    char_to_replace = char.replace('space(', '')
                    char_to_replace = char_to_replace.replace(')', '')
                    print("char_to_replace: ", char_to_replace)
                    char_escaped = re.escape(char_to_replace).replace(r'\+', '+')
                    print("char_escaped: ", char_escaped)
                    dfg = dfg.applymap(lambda x: re.sub(char_escaped, ' ', str(x)))
            elif char.startswith("pref("):
                match = re.match(r"pref\((\d+),\s*(.+)\)", char)
                if match:
                    num_chars = int(match.group(1))
                    column_name = match.group(2).strip()
                    if num_chars > 0:
                        dfg[column_name] = dfg[column_name].apply(lambda x: x[num_chars:] if isinstance(x, str) else x)
            elif char.startswith("sub("):
                match = re.match(r"sub\((\d+),\s*(.+)\)", char)
                if match:
                    num_chars = int(match.group(1))
                    column_name = match.group(2).strip()
                    if num_chars > 0:
                        dfg[column_name] = dfg[column_name].apply(lambda x: x[:-num_chars] if isinstance(x, str) else x)
            elif char.startswith("drow("):
                  print("entro a row")
                  dfg = dfg.dropna(how = 'all')
            elif char.startswith("merge("):
                match = re.match(r"merge\((.+),\s*(.+)\),\s*(.+)\)", char)
                if match:
                    column_name1 = match.group(1).strip()
                    column_name2 = match.group(2).strip()
                    new_column_name = match.group(3).strip()
                    dfg[new_column_name] = dfg[column_name1] + dfg[column_name]
            elif char.startswith("join("):
                print("entro join")
                match = re.match(r"join\((.+),\s*(.+)\)", char)
                if match:
                    print("entro if")
                    column_name1 = match.group(1).strip()
                    column_name2 = match.group(2).strip()

                    mask = dfg[column_name2] == ''
                    dfg.loc[mask, column_name2] = dfg.loc[mask, column_name1]

            elif char.startswith("may("):
                match = re.match(r"may\((.*)\)", char)
                if match:
                    column_name = match.group(1).strip()
                    if column_name:
                        try:
                            dfg[column_name] = dfg[column_name].apply(lambda x: x.title() if isinstance(x, str) else x)
                        except:
                            break
                    else:
                        dfg = dfg.applymap(lambda x: x.title() if isinstance(x, str) else x)
            elif char.startswith("min("):
                match = re.match(r"min\((.*)\)", char)
                if match:
                    column_name = match.group(1).strip()
                    if column_name:
                        try:
                           dfg[column_name] = dfg[column_name].str.lower()
                        except:
                            break
                    else:
                        dfg = dfg.applymap(lambda x: x.lower() if isinstance(x, str) else x)



            elif char.startswith("div("):
                match = re.match(r"div\((.+),\s*(\d+),\s*(.+)\)", char)
                if match:
                    column_name = match.group(1).strip()
                    num_chars = int(match.group(2))
                    new_column_name = match.group(3).strip()
                    dfg[new_column_name] = dfg[column_name].apply(lambda x: x[num_chars:] if isinstance(x, str) else x)
                    dfg[column_name] = dfg[column_name].apply(lambda x: x[:num_chars] if isinstance(x, str) else x)
            elif char.startswith("del.contains("):
                match = re.match(r"del\.contains\((.+),\s*(.+)\)", char)
                if match:
                    print("Entro al if de del.contains")
                    column_name = match.group(2).strip()
                    char_to_remove = match.group(1).strip()
                    char_escaped = re.escape(char_to_remove)
                    print("column_name: ", column_name)
                    print("char_to_remove: ", char_to_remove)
                    print("char_escaped: ", char_escaped)
                    dfg[column_name] = dfg[column_name].str.replace(f".*{char_escaped}.*", "", regex=True)
            elif char.startswith("del.ex("):
                match = re.match(r"del\.ex\((.+),\s*(.+)\)", char)
                if match:
                    print("Entro al if de del.ex")
                    column_name = match.group(2).strip()
                    char_to_keep = match.group(1).strip()
                    char_escaped = re.escape(char_to_keep)
                    print("column_name: ", column_name)
                    print("char_to_keep: ", char_to_keep)
                    print("char_escaped: ", char_escaped)
                    dfg[column_name] = dfg[column_name].str.replace(f"^(?!.*{char_escaped}).*$", "", regex=True)

            else:
                char_escaped = re.escape(char).replace(r'\+', '\\+')
                dfg = dfg.replace(char_escaped, "", regex=True)
            form()
            char = input("Introduce tu Formula: ")
        except re.error:
            print("Error: Expresión regular inválida. Introduce otro carácter.")
            print("")
            form()
            char = input("Introduce tu Formula: ")

def credentials():
    urlg = "https://github.com/OloAlexi/Olozf/raw/main/credentials.json"
    response = requests.get(urlg)
    content = response.content

    temp_file = tempfile.NamedTemporaryFile(delete=False)
    temp_file.write(content)
    temp_file.close()

    temp_file_path = temp_file.name

    global gc
    global spreadsheet

    gc = gspread.service_account(filename=temp_file_path)
    spreadsheet = gc.open_by_url(url_input)

def analyze_and_set_column_formats(df):
    for column in df.columns:
        column_data = df[column]
        column_data_type = column_data.apply(type).unique()

        if len(column_data_type) == 1:
            if column_data_type[0] == str:
                df[column] = column_data.astype(str)
            elif column_data_type[0] in [int, float]:
                df[column] = pd.to_numeric(column_data)
            elif column_data_type[0] == pd.Timestamp:
                df[column] = pd.to_datetime(column_data)
            elif column_data_type[0] == pd.Timedelta:
                df[column] = pd.to_timedelta(column_data)

    return df

def charge():
    global gc
    global spreadsheet
    clear()
    print("Cargando...")
    try:
        worksheet = spreadsheet.worksheet(sheet)
        backup_data = worksheet.get_all_values()
        worksheet.clear()

        dfg_cleaned = analyze_and_set_column_formats(dfg)

        data = [dfg_cleaned.columns.tolist()] + dfg_cleaned.values.tolist()
        num_rows, num_cols = dfg_cleaned.shape

        cell_range = f'A1:{chr(ord("A") + num_cols - 1)}{num_rows + 1}'

        if "{" in cell_range or "}" in cell_range or "~" in cell_range or "\x80" in cell_range or "\x7f" in cell_range or "\x94" in cell_range:
            cell_range = cell_range.replace("{", "").replace("}", "").replace("~", "").replace("\x80", "").replace("\x94", "")
        worksheet.update(cell_range, data)

        clear()
        print("Datos limpiados y cargados")

    except Exception as e:
        clear()
        print("Error")
        print("Restaurando datos de la copia de seguridad...")
        if backup_data:
            worksheet.clear()
            worksheet.update('A1', backup_data)
            clear()
            print("Error")
            print("Copia de seguridad restaurada")
            print("")
            print("Avisar a Alexi, mandale el siguiente codigo")
            print("")
            print("")
            print("Codigo del error: ", str(e))
            print("")
            print("dfg_cleaned.shape: ", dfg_cleaned.shape)
            print("dfg.shape: ", dfg.shape)
            print("cell_range: ", cell_range)
            print("num_rows: ", num_rows)
            print("num_cols: ", num_cols)
        else:
            clear()
            print("No se encontró una copia de seguridad")

Main()
    #By Alexi Marin